In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras import Model

from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb

In [2]:
import os
import numpy as np
from PIL import Image

FACES_DIR = "faces"
SMILE_LIST = "lists/SMILE_list.txt"
NONSMILE_LIST = "lists/NON-SMILE_list.txt"


with open(SMILE_LIST, "r") as f:
    smile_files = [line.strip() for line in f if line.strip() != ""]

with open(NONSMILE_LIST, "r") as f:
    nonsmile_files = [line.strip() for line in f if line.strip() != ""]

print("Ejemplos SMILE_list:", smile_files[:5])
print("Ejemplos NONSMILE_list:", nonsmile_files[:5])


def safe_load(path):
    try:
        return np.array(Image.open(path)) / 255.0
    except Exception as e:
        print(f"⚠️ Error leyendo {path}: {e}")
        return None


X = []
y = []

for fname in smile_files:
    ppm_name = fname.replace(".jpg", ".ppm")  
    path = os.path.join(FACES_DIR, ppm_name)
    if os.path.exists(path):
        img = safe_load(path)
        if img is not None:
            X.append(img)
            y.append("smile")
    else:
        print("No existe:", path)

for fname in nonsmile_files:
    ppm_name = fname.replace(".jpg", ".ppm")  
    path = os.path.join(FACES_DIR, ppm_name)
    if os.path.exists(path):
        img = safe_load(path)
        if img is not None:
            X.append(img)
            y.append("non-smile")
    else:
        print("No existe:", path)

features = np.array(X)
labels = np.array(y)

print("features shape:", features.shape)
print("labels shape:", labels.shape)

Ejemplos SMILE_list: ['James_Jones_0001.jpg', 'James_Kelly_0009.jpg', 'James_McPherson_0001.jpg', 'James_Watt_0001.jpg', 'Jamie_Carey_0001.jpg']
Ejemplos NONSMILE_list: ['Aaron_Eckhart_0001.jpg', 'Aaron_Patterson_0001.jpg', 'Abdel_Nasser_Assidi_0002.jpg', 'Abdoulaye_Wade_0004.jpg', 'Abdulaziz_Kamilov_0001.jpg']
No existe: faces\listt.txt
No existe: faces\SMILE_list.txt
features shape: (1203, 64, 64, 3)
labels shape: (1203,)


In [3]:
print("Ejemplos SMILE_list:", smile_files[:5])
print("Ejemplos NONSMILE_list:", nonsmile_files[:5])


Ejemplos SMILE_list: ['James_Jones_0001.jpg', 'James_Kelly_0009.jpg', 'James_McPherson_0001.jpg', 'James_Watt_0001.jpg', 'Jamie_Carey_0001.jpg']
Ejemplos NONSMILE_list: ['Aaron_Eckhart_0001.jpg', 'Aaron_Patterson_0001.jpg', 'Abdel_Nasser_Assidi_0002.jpg', 'Abdoulaye_Wade_0004.jpg', 'Abdulaziz_Kamilov_0001.jpg']


In [4]:
print(labels.shape)
print(features.shape)

(1203,)
(1203, 64, 64, 3)


In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

labels_int = le.fit_transform(labels.ravel())

In [6]:
print(labels[0])
print(labels_int[0])

smile
1


In [7]:
labels = tf.keras.utils.to_categorical(labels_int, num_classes = 2)

In [8]:
n_samples = len(features)
data = features.reshape((n_samples, -1))

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, labels_int, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(962, 12288)

In [11]:
model = xgb.XGBClassifier(objective = "multi:softmax",
                            num_class = 2,
                            eval_metric = "mlogloss",
                            use_label_encoder = False,
                            n_estimators = 100,
                            learning_rate = 0.1,
                            max_depth = 5 
)

In [12]:
model.fit(X_train, y_train)

c:\Users\danis\anaconda3\envs\mv-tec\Lib\site-packages\xgboost\training.py:199: UserWarning: [23:05:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=2, ...)

In [13]:
y_pred_int = model.predict(X_test)

In [14]:
y_pred_int.shape

(241,)

In [15]:
y_pred_labels = le.inverse_transform(y_pred_int)
y_test_labels = le.inverse_transform(y_test)
print("Ejemplos y_test_labels:", y_test_labels[:5])
print("Ejemplos y_pred_labels:", y_pred_labels[:5])

Ejemplos y_test_labels: ['non-smile' 'smile' 'smile' 'non-smile' 'smile']
Ejemplos y_pred_labels: ['non-smile' 'smile' 'smile' 'non-smile' 'non-smile']


In [16]:
pd.crosstab(y_test_labels, y_pred_labels, rownames=['True labels'], colnames=['Predicted labels'])

Predicted labels,non-smile,smile
True labels,,
non-smile,100,16
smile,19,106


In [18]:
accuracy = accuracy_score(y_test, y_pred_int)
print("Accuracy:", accuracy)

Accuracy: 0.8547717842323651


In [19]:
print(classification_report(y_test_labels, y_pred_labels))

              precision    recall  f1-score   support

   non-smile       0.84      0.86      0.85       116
       smile       0.87      0.85      0.86       125

    accuracy                           0.85       241
   macro avg       0.85      0.86      0.85       241
weighted avg       0.86      0.85      0.85       241

